<img src="Archivos/miad4.png" width=800x>

## Interactividad usando `panel`

En este taller podrás poner en práctica los conceptos aprendidos acerca del uso del paquete `panel`. Usarás `panel` con el fin de crear dashboards como herramienta de apoyo al momento de resolver problemas específicos en un contexto de Analytics.

Instrucciones y condiciones generales: Esta es una actividad calificada y, por lo tanto, debe ser resuelta individualmente.

### Al desarrollar este taller podrás verificar tu progreso en:

1. Programar la funcionalidad de un dashboard en `panel`.
2. Agregar gráficos y widgets para hacer visualizaciones agradables en `panel`.
3. Personalizar dashobards para generar distintas visualizaciones interactivas en `panel`.

In [3]:
import panel as pn
import plotly.graph_objects as go
import pandas as pd
pn.extension('plotly')

## Pregunta 1

Con la base de datos *COVID-19.csv* crea un dashboard en `panel` que:

1. Permita crear un diagrama de barras horizontal que muestre los casos de COVID-19 de acuerdo a la selección de los siguentes criterios:<br />
    a. Casos o muertes.<br />
    b. Periodo: Histórico, Últimos 7 días y Últimas 24 horas.<br />
    Pista: utilice un control de tipo `RadioButtonGroup` para cada uno de los dos criterios.<br />
2. Contenga un diagrama de barras horizontal de las muertes por millón de personas.
3. Permita visualizar a tráves de un diagrama de barras horizontal los casos en cada una de las regiones de la Organización Munidal de la Salud.

Nota: a excpeción de las regiones, muestre los 10 mayores valores en cada gráfico.

| <center>Variable</center> |<center>Descripción</center> | 
| :-- | :-- | 
| Name | País |
| WHO Region | Región de la OMS |
| Cases - cumulative total | Total de casos hasta el momento |
| Cases - cumulative total per 1 million population | Número de casos acumulado por millón de personas |
| Cases - newly reported in last 7 days | Número de casos nuevos en los últimos 7 días |
| Cases - newly reported in last 24 hours | Número de casos nuevos en las últimas 24 horas |
| Deaths - cumulative total | Total de muertes hasta el momento |
| Deaths - cumulative total per 1 million population | Número de muertes acumulado por millón de personas |
| Deaths - newly reported in last 7 days | Número de muertes nuevas en los últimos 7 días |
| Deaths - newly reported in last 24 hours | Número de muertes nuevas en las últimas 24 horas |

In [4]:
# YOUR CODE HERE
#primero vamos a importar la base:
datos = pd.read_csv('C:/Users/STEPHANIE/Desktop/Talleres Herramientas/S6 - Visualizacion e interactividad/S6.TA2/Archivos/COVID-19.csv')

datos = pd.DataFrame(datos)
datos.head()

#raise NotImplementedError()

,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 1 million population,Cases - newly reported in last 7 days,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 1 million population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 24 hours,Transmission Classification
0,United States of America,Americas,26547977,80204.73,871365,129961,455735,1376.83,22562,3549,Community transmission
1,India,South-East Asia,10826363,7845.17,80180,12059,154996,112.32,722,78,Clusters of cases
2,Brazil,Americas,9447165,44444.82,328652,50872,230034,1082.21,7368,1239,Community transmission
3,Russian Federation,Europe,3967281,27185.36,116842,16048,76661,525.31,3479,432,Clusters of cases
4,The United Kingdom,Europe,3929839,57888.79,133747,18262,112092,1651.18,6521,828,Community transmission


In [5]:
#titulo del tablero
titulo = '# Tablero de control Covid-19'


#primero voy a  hacer los botones
#creo el boton de casos o muertes
seleccion_cm = pn.widgets.RadioButtonGroup(name='Casos o Muertes', options=['Casos', 'Muertes'], button_type='success')
#creo el boton de Periodo
seleccion_periodo = pn.widgets.RadioButtonGroup(name='Periodo', options=["Historico", 'Ultimos 7 dias', 'Ultimas 24 horas'], button_type='success')

In [6]:
def figura_muertes_por_millon():
    df = datos
    df.sort_values(by=["Deaths - cumulative total per 1 million population"], inplace=True, ascending=False)
    
    fig = go.Figure(go.Bar(
                            x=df["Deaths - cumulative total per 1 million population"],
                            y=df["Name"],
                            marker_color='rgb(26, 118, 255)',                      # Color de las barras.
                            orientation='h'                                        # Orientación de las barras. En este caso horizontal.
                          )
                   )
    
                    
    fig.update_layout(
                        barmode='stack',                                           # Tipo de barras. En este caso apiliadas.
                        yaxis_title="Region OMS",
                        xaxis_title="Cantidad de muertes",
                        title= 'Cantidad de muertes por millon por ciudad',
                        xaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje x.
                        yaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje y.
                        plot_bgcolor='white',                                      # Modificamos el color del fondo del gráfico.
                        showlegend=False,                                          # Ocultamos la leyenda.
                        yaxis_range=[9.5, -.5]                                     # Mostramos los 10 primeros datos.
                     )
    
    return fig

In [7]:
#primera grafica
# Le decimos a panel que nuestro gráfico depende de cambios en el párametro seleccion.
@pn.depends(seleccion_periodo.param.value,seleccion_cm.param.value)

# Definimos la función que se encarga de la interactividad del tablero de control.
def funcion_interactiva_casos(seleccion_periodo,seleccion_cm): 
    #titulo_grafico
    if seleccion_cm == "Casos":
        titulo = 'Cantidad de casos reportados en '+ str(seleccion_periodo)
    else:
        titulo = "Cantidad de muertes por millon por ciudad"
    
    if seleccion_cm == "Casos":
        # Cargamos los datos.
        df = datos 
        # Agrupamos los datos por region OMS.
        df=df.groupby('WHO Region').sum().reset_index()

        #definimos el eje x

        if seleccion_periodo == "Historico":
            eje_x = "Cases - cumulative total"
        elif seleccion_periodo == "Ultimos 7 dias":
            eje_x = "Cases - newly reported in last 7 days"
        else:
            eje_x="Cases - newly reported in last 24 hours"


        # Ordenamos los datos de mayor a menor.
        df.sort_values(by=[eje_x], inplace=True, ascending=False)

        # Definimos y retornamos la figura.
        fig = go.Figure(go.Bar(
                                x=df[eje_x],
                                y=df["WHO Region"],
                                marker_color='rgb(26, 118, 255)',                      # Color de las barras.
                                orientation='h'                                        # Orientación de las barras. En este caso horizontal.
                              )
                       )


        fig.update_layout(
                            barmode='stack',                                           # Tipo de barras. En este caso apiliadas.
                            yaxis_title="Region OMS",
                            xaxis_title="Cantidad de muertes",
                            title= titulo,
                            xaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje x.
                            yaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje y.
                            plot_bgcolor='white',                                      # Modificamos el color del fondo del gráfico.
                            showlegend=False,                                          # Ocultamos la leyenda.
                            yaxis_range=[9.5, -.5]                                     # Mostramos los 10 primeros datos.
                         )
    else:
        fig=figura_muertes_por_millon()
    
    return fig

In [8]:
tablero_control = pn.Row(pn.Column(titulo, seleccion_cm,seleccion_periodo), funcion_interactiva_casos)

In [9]:
tablero_control.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] RadioButtonGroup(button_type='success', name='Casos o Muertes', options=['Casos', 'Muertes'], value='Casos')
        [2] RadioButtonGroup(button_type='success', name='Periodo', options=['Historico', ...], value='Historico')
    [1] ParamFunction(function)

In [10]:
tablero_control.show()

Launching server at http://localhost:58792


In [4]:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
#raise NotImplementedError()

In [99]:
# YOUR CODE HERE
#raise NotImplementedError()

## Créditos

**Autor(es)**: Juan David Reyes Jaimes, Diego Alejandro Cely Gomez

**Última actualización:** 26/08/2021